## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Для начала, вызовем библиотеку pandas.

In [1]:
import pandas as pd

Откроем файл '/datasets/data.csv', и прочтём первые 10 строк.

In [2]:
credit_info = pd.read_csv('/datasets/data.csv')
credit_info.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В предоставленной таблице 12 столбцов:
- сhildren - количество детей в семье
- days_employed - общий трудовой стаж в днях
- dob_years - возраст клиента в годах
- education - уровень образования клиента
- education_id - идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Посмотрим на общую информацию о таблице.

In [3]:
credit_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Сразу видно, что в столбцах 'days_employed' и 'total_income' есть пропуски. Причём равное количество.

Посмотрим внимательно на данные в столбце 'children'.

In [4]:
credit_info['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Все данные кажутся реальными, кроме '20' и '-1'. Можно предположить, что изначально данные имели вид '2' и '1'.

Посмотрим на столбец 'days_employed'.

In [5]:
credit_info['days_employed'].value_counts()

-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64

Многие данные отрицательные. Также мы уже знаем, что в этом столбце есть пропуски. Посмотрим, есть ли здесь вообще положительные значения.

In [6]:
credit_info[credit_info['days_employed'] > 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


На первый взгляд положительные значения в 'days_employed' есть только у пенсионеров. Cредний стаж пенсионеров достигает 1000 лет!

In [7]:
credit_info.groupby('income_type').sum()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
income_type,,,,,,,
безработный,1,7.328273e+05,76,1,1,1,2.626795e+05
в декрете,2,-3.296760e+03,39,1,0,1,5.382913e+04
госслужащий,908,-4.460665e+06,59289,980,1251,86,2.242186e+08
компаньон,3044,-9.664447e+06,201862,3638,5250,376,9.264647e+08
пенсионер,509,1.256707e+09,227747,3524,3801,216,4.721299e+08
предприниматель,0,-5.208481e+02,85,0,1,0,4.991631e+05
сотрудник,7136,-2.329756e+07,442770,9447,10626,1061,1.616062e+09
студент,0,-5.787516e+02,22,0,4,0,9.820163e+04


Положительные значения в 'days_employed' есть не только у пенсионеров, но и у безработных.

In [8]:
credit_info.loc[credit_info.loc[:, 'income_type'] == 'безработный']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,395302.838654,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


В таблице есть два безработных, стаж которых приближается к 1000 годам.

Посмотрим на столбец 'dob_years'.

In [9]:
credit_info['dob_years'].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

В столбце выделяются клиенты с возрастом '0'. Посмотрим на остальные данные клиентов с этим возрастом.

In [10]:
credit_info[credit_info['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,жилье
20462,0,338734.868540,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,331741.271455,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,-108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


Возраст '0' указан у совершенно разных групп клиентов, в том числе у пенсионеров.

Посмотрим на столбец 'education'.

In [11]:
credit_info['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

В этом столбце присутствует множество дупликатов, но можно выделить 5 групп:
- начальное
- среднее
- неоконченное высшее
- высшее
- учёная степень

Посмотрим на столбец 'education_id'.

In [12]:
credit_info['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

В этом столбце представлены 5 числовых идентификаторов (0,1,2,3,4), соответствующих 5 группам образований из предыдущего столбца.

Посмотрим на столбец 'family_status'.

In [13]:
credit_info['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

В этом столбце указаны 5 групп семейного положения.

Посмотрим на столбец 'family_status_id'.

In [14]:
credit_info['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Как и в случае c 'education_id', в этом столбце представлены 5 числовых идентификаторов (0,1,2,3,4), соответствующих 5 группам образований из предыдущего столбца.

Теперь столбец 'gender'.

In [15]:
credit_info['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Всё стандартно, кроме одного значения - XNA.

Столбец 'income_type'.

In [16]:
credit_info['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Аномалий не замечено. Теперь 'debt'.

In [17]:
credit_info['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

Аномалий нет.

Столбец 'total_income' мы смотреть не будет: в нём огромное количество уникальных значений. К тому же мы уже знаем, что в нём есть пропуски.

И последний столбец - 'purpose'.

In [18]:
credit_info['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Здесь можно заметить похожие цели для взятия кредита. Придётся выделить леммы.

### Вывод

В предоставленной нам таблице много неточной информации:
- наличие дупликатов в столбцах 'children', 'gender' и 'education'
- данные в 'days_employed' и 'total_income' имеют отрицательное значение
- данные в 'days_employed' и 'total_income' имеют вид дробных чисел
- значения 'days_employed' для пенсионеров и безработных нереально высоки
- пропуски в 'days_employed' и 'total_income'

Перед анализом нам необходимо привести данную таблицу в приемлимый вид.

### Шаг 2. Предобработка данных

### Обработка пропусков

Изучив общие даннные таблицы, мы узнали о наличие равного количества пропусков в столбцах 'days_emplayed' и 'total_income'.

Посмотрим на первые 10 пропусков в столбце 'days_employed'.

In [19]:
credit_info[credit_info['days_employed'].isnull()].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

In [20]:
credit_info[credit_info['days_employed'].isnull()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Начнём с простого: заполнения пропусков в 'days_employed'. Данные из этого столбца нам не понадобятся для выполнения последующих заданий, так что пропуски мы заполним нулями.

In [21]:
credit_info['days_employed'] = credit_info['days_employed'].fillna(0)

Проверим наличие пропусков в столбце 'days_employed'.

In [22]:
credit_info[credit_info['days_employed'].isnull()].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [23]:
credit_info[credit_info['days_employed'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,0.0,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,0.0,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,0.0,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,0.0,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,0.0,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,0.0,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,0.0,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,0.0,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,0.0,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Теперь к 'total_income'. Мы не можем просто заполнить пропуски нулями, т.к. собираемся определить зависимость между уровнем дохода и возвратом кредита в срок. Нужно посчитать медианы дохода для каждого типа занятости, коих у нас 8:
- сотрудник
- компаньон
- пенсионер
- госслужащий
- безработный
- предприниматель
- студент
- в декрете

Напишем переменную 'income_types', записав в неё все группы занятости, и 'income_types_grouped_median', считающую медианы дохода для каждой группы.

In [24]:
income_types = credit_info['income_type'].unique().tolist()
income_types_grouped_median = credit_info.groupby('income_type')['total_income'].median()

In [25]:
income_types

['сотрудник',
 'пенсионер',
 'компаньон',
 'госслужащий',
 'безработный',
 'предприниматель',
 'студент',
 'в декрете']

In [26]:
income_types_grouped_median

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Теперь напишем цикл для заполнения пропусков в 'total_income'.

In [27]:
for element in income_types:
    credit_info.loc[credit_info['income_type'] == element, 'total_income'] = credit_info.loc[credit_info['income_type'] == element, 'total_income'].fillna(income_types_grouped_median[element])

In [28]:
credit_info[credit_info['total_income'].isnull()].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Хм, .isnull() не находит пропуски в 'total_income'. Теперь вызовем строку 41, в которой был пропуск в 'total_income', и посмотрим, как он заполнился.

In [29]:
credit_info.loc[41]

children                                           0
days_employed                                      0
dob_years                                         50
education                                    среднее
education_id                                       1
family_status                        женат / замужем
family_status_id                                   0
gender                                             F
income_type                              госслужащий
debt                                               0
total_income                                  150448
purpose             сделка с подержанным автомобилем
Name: 41, dtype: object

В 'total_income' клиента № 41 теперь значение 150448, равное медиане для группы занятости "госслужащий".

### Вывод

В ходе изучения таблицы 'credit_info' были обнаружены пропуски в столбцах 'days_employed' (трудовой стаж в днях) и 'total_income' (ежемесячный доход).

Пропуски были обнаружены у совершенно разных групп клиентов. Причём пропуски в этих двух столбцах сопровождали друг друга.
Причины возникновения пропусков:
- банк не смог добыть эти данные
- утеря данных связана с технической ошибкой в системе кредитного отдела банка

Пропуски в столбике 'days_employed' были заполнены нулями, т.к. данные из этого столбца нам не понадобятся для решения последующих задач. Со столбцом 'total_income' наоборот: нам нужно определить зависимость между доходом клиента и вероятностью возврата кредита в срок. Поэтому было решено определить медианы дохода для каждой группы занятости, и заполнить ими пропуски в 'total_income'.

### Замена типа данных

Нужно перевести тип данных в 'days_employed' и 'total_income' из float в int.

In [30]:
credit_info['days_employed'] = credit_info['days_employed'].astype('int')

In [31]:
credit_info['total_income'] = credit_info['total_income'].astype('int')

In [32]:
credit_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Теперь данные в 'days_employed' и 'total_income' имеют тип int64, с которым проще выполнять вычисления.

### Обработка дубликатов

Изучив данные предоставленной нам таблицы, мы узнали о наличии дубликатов в столбцах 'children' и 'education'. Для начала приведём данные 'education' в нижний регистр.

In [33]:
credit_info['education'] = credit_info['education'].str.lower()
credit_info['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Теперь заменим значения '-1' и '20' из 'children' на '1' и '2' соответственно.

In [34]:
credit_info['children'] = credit_info['children'].replace(-1, 1)
credit_info['children'] = credit_info['children'].replace(20, 1)
credit_info['children'].value_counts()

0    14149
1     4941
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Проверяем всю таблицу на наличие дубликатов.

In [35]:
credit_info.duplicated().sum()

71

Используем метод drop_duplicates для удаления дубликатов.

In [36]:
credit_info = credit_info.drop_duplicates().reset_index(drop = True)
credit_info.duplicated().sum()

0

### Вывод

Перед поиском и удалением дубликатов нам нужно было убрать аномальные данные из столбцов 'children' и 'education'. Для 'children' использовался метод .replace(), т.к. нужно было заменить всего два значения: '-1' и '20'. Для 'education' использовался метод str.lower(), вернувший строки колонки к нижнему регистру. После этого был использован метод .drop_duplicates() для удаления всех дубликатов в таблице. Причина появления дубликатов - ошибка при сборе данных клиентов.

### Лемматизация

Для начала нужно импортировать библиотеку pymystem3.

In [37]:
from pymystem3 import Mystem
m = Mystem()

Теперь пишем функцию для выделения лемм.

In [38]:
def lemmatizator(i):
    lemmas = m.lemmatize(i)
    return lemmas

А теперь применяем её к столбцу 'purpose', и полученны результат переводим в новую колонку 'purpose_lemmas'.

In [39]:
credit_info['purpose_lemmas'] = credit_info['purpose'].apply(lemmatizator)
credit_info['purpose_lemmas'].head(10)

0                      [покупка,  , жилье, \n]
1            [приобретение,  , автомобиль, \n]
2                      [покупка,  , жилье, \n]
3         [дополнительный,  , образование, \n]
4                    [сыграть,  , свадьба, \n]
5                      [покупка,  , жилье, \n]
6               [операция,  , с,  , жилье, \n]
7                            [образование, \n]
8          [на,  , проведение,  , свадьба, \n]
9    [покупка,  , жилье,  , для,  , семья, \n]
Name: purpose_lemmas, dtype: object

В колонке 'purpose_lemmas' много лишних слов, которые будут мешать. Создадим новую колонку, в которой будут приведены категории целей кредитов.

In [40]:
def purpose_type(purpose_lemmas):
    try:
        if 'жилье' in purpose_lemmas:
            return 'недвижимость'
        if 'недвижимость' in purpose_lemmas:
            return 'недвижимость'
        if 'квартира' in purpose_lemmas:
            return 'недвижимость'
        if 'свадьба' in purpose_lemmas:
            return 'свадьба'
        if 'автомобиль' in purpose_lemmas:
            return 'автомобиль'
        if 'образование' in purpose_lemmas:
            return 'образование'
    except:
        return 'другое'
credit_info['purpose_type'] = credit_info['purpose_lemmas'].apply(purpose_type)
credit_info['purpose_type'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_type, dtype: int64

Теперь причислим группам целей соответствующие ID.

In [41]:
def purpose_type_id(purpose_type):
    if 'недвижимость' in purpose_type:
            return '1'
    if 'автомобиль' in purpose_type:
            return '2'        
    if 'образование' in purpose_type:
            return '3'
    else:
            return '4'
credit_info['purpose_type_id'] = credit_info['purpose_type'].apply(purpose_type_id)
credit_info['purpose_type_id'].value_counts()

1    10811
2     4306
3     4013
4     2324
Name: purpose_type_id, dtype: int64

In [42]:
credit_info.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_type,purpose_type_id
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,1
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,2
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,1
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,3
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,4
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость,1
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,1
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование,3
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,4
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость,1


### Вывод

Для проведения лемматизации к колонке 'purpose' была применена функция lemmatizator, а результат был записан в колонку 'purpose_lemmas'. После изучения колонки 'purpose_lemmas' были выделены 4 группы целей кредитов:
- недвижимость
- автомобиль
- образование
- свадьба

Далее была составлена функция 'purpose_type', которая проверяет колонку 'purpose_lemmas' на наличие ключевых слов, и при совпадении, определяет цель кредита к определённой группе.

### Категоризация данных

Нам нужно определить, влияют ли на возврат кредита в срок следующие критерии:
- наличие детей
- семейное положение
- уровень дохода
- цель получения кредита

По целям кредитов мы уже категоризовали, поэтому сейчас мы будем категоризировать данные по остальным критериям.

Начнём с детей. Разделим клиентов на группы по количеству детей:
-  0 - бездетный
-  от 1 до 3 - малодетный
-  больше 3 - многодетный

In [43]:
def children_type(row):
    children = row['children']
    if children == 0:
        return 'бездетный'
    if children >= 1 and children < 3:
        return 'малодетный'
    else:
        return 'многодетный'

In [44]:
credit_info['children_type'] = credit_info.apply(children_type, axis = 1)

In [45]:
credit_info['children_type'].value_counts()

бездетный      14091
малодетный      6983
многодетный      380
Name: children_type, dtype: int64

Добавим id для каждой группы.

In [46]:
def children_type_id(children_type):
    if 'бездетный' in children_type:
            return '1'
    if 'малодетный' in children_type:
            return '2'
    else:
            return '3'
credit_info['children_type_id'] = credit_info['children_type'].apply(children_type_id)
credit_info['children_type_id'].value_counts()

1    14091
2     6983
3      380
Name: children_type_id, dtype: int64

Теперь семейное положение. У нас уже есть несколько групп:
- женат / замужем
- гражданский брак
- Не женат / не замужем
- в разводе
- вдовец / вдова

Их можно сгруппировать в две группы:
- одинокие
- в отношениях

In [47]:
def relationship_status(row):
    relationship = row['family_status_id']
    if relationship >= 0 and relationship <= 1:
        return 'в отношениях'
    else:
        return 'одинокие'

In [48]:
credit_info['relationship_status'] = credit_info.apply(relationship_status, axis = 1)

In [49]:
credit_info['relationship_status'].value_counts()

в отношениях    16490
одинокие         4964
Name: relationship_status, dtype: int64

Теперь причислим каждой группе ID.

In [50]:
def relationship_status_id(relationship_status):
    if 'в отношениях' in relationship_status:
            return '1'
    else:
            return '2'
credit_info['relationship_status_id'] = credit_info['relationship_status'].apply(relationship_status_id)
credit_info['relationship_status_id'].value_counts()

1    16490
2     4964
Name: relationship_status_id, dtype: int64

Теперь категоризация по уровню дохода. Клиентов мы разделим на 4 группы по уровню дохода в месяц:
- меньше 60 000 - низкий доход
- меньше 120 000 - средний доход
- меньше 1 000 000 - высокий доход
- больше 1 000 000 - миллионер

In [51]:
def income_level(row):
    level = row['total_income']
    if level <= 60000:
        return 'низкий доход'
    if level <= 120000 and level > 50000: 
        return 'средний доход'
    if level <= 1000000 and level > 120000:
        return 'высокий доход'
    else:
        return 'миллионер'

In [52]:
credit_info['income_level'] = credit_info.apply(income_level, axis = 1)

In [53]:
credit_info['income_level'].value_counts()

высокий доход    14198
средний доход     6425
низкий доход       806
миллионер           25
Name: income_level, dtype: int64

Причислим id группам по доходу.

In [54]:
def income_level_id(income_level):
    if 'низкий доход' in income_level:
            return '1'
    if 'средний доход' in income_level:
            return '2'
    if 'высокий доход' in income_level:
            return '3'
    else:
            return '4'
credit_info['income_level_id'] = credit_info['income_level'].apply(income_level_id)
credit_info['income_level_id'].value_counts()

3    14198
2     6425
1      806
4       25
Name: income_level_id, dtype: int64

Не забудем перевести новые строчки с id в int64.

In [55]:
credit_info['purpose_type_id'] = credit_info['purpose_type_id'].astype('int')
credit_info['children_type_id'] = credit_info['children_type_id'].astype('int')
credit_info['relationship_status_id'] = credit_info['relationship_status_id'].astype('int')
credit_info['income_level_id'] = credit_info['income_level_id'].astype('int')

In [56]:
credit_info.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,...,purpose,purpose_lemmas,purpose_type,purpose_type_id,children_type,children_type_id,relationship_status,relationship_status_id,income_level,income_level_id
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,...,покупка жилья,"[покупка, , жилье, \n]",недвижимость,1,малодетный,2,в отношениях,1,высокий доход,3
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,...,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,2,малодетный,2,в отношениях,1,средний доход,2
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,...,покупка жилья,"[покупка, , жилье, \n]",недвижимость,1,бездетный,1,в отношениях,1,высокий доход,3
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,...,дополнительное образование,"[дополнительный, , образование, \n]",образование,3,многодетный,3,в отношениях,1,высокий доход,3
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,...,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,4,бездетный,1,в отношениях,1,высокий доход,3
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,...,покупка жилья,"[покупка, , жилье, \n]",недвижимость,1,бездетный,1,в отношениях,1,высокий доход,3
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,...,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,1,бездетный,1,в отношениях,1,высокий доход,3
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,...,образование,"[образование, \n]",образование,3,бездетный,1,в отношениях,1,высокий доход,3
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,...,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,4,малодетный,2,в отношениях,1,средний доход,2
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,...,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость,1,бездетный,1,в отношениях,1,высокий доход,3


### Вывод

Мы категоризовали данные по следующим критериям:
- количество детей
- семейное положение
- уровень дохода

Было решено использовать критерий 'количество детей' вместо 'наличие детей', т.к. разное количество детей может по разному влиять на вероятность возврата кредита.

Клиенты были разделены на две группы по семейному положению: одиночек, и состоящих в отношениях. Такой выбор критериев обусловлен предположением, что клиенту проще вернуть кредит в срок, если у него есть возможноть разделить это бремя со своим партнёром, не зависимо от правового статуса их отношений.

Для уровня дохода были выбраны следующие критерии:
- меньше 60 000 - низкий доход
- меньше 120 000 - средний доход
- меньше 1 000 000 - высокий доход
- больше 1 000 000 - миллионер

Такой выбор был сделан на основании нескольких публикаций, утверждавших, что для причисления себя к среднему классу в регионах РФ нужно иметь месячный доход быольше 60 000 рублей. А в Москве - больше 120 000 рублей.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [57]:
def children_debt(row):
    if row['children_type_id'] == 1 and row['debt'] == 0:
        return 'бездетный без долга'
    if row['children_type_id'] == 1 and row['debt'] == 1:
        return 'бездетный с долгом'
    if row['children_type_id'] == 2 and row['debt'] == 0:
        return 'малодетный без долга'
    if row['children_type_id'] == 2 and row['debt'] == 1:
        return 'малодетный с долгом'
    if row['children_type_id'] == 3 and row['debt'] == 0:
        return 'многодетный без долга'
    if row['children_type_id'] == 3 and row['debt'] == 1:
        return 'многодетный с долгом'
credit_info['children_debt'] = credit_info.apply(children_debt, axis=1)
credit_info['children_debt'].value_counts()

бездетный без долга      13028
малодетный без долга      6336
бездетный с долгом        1063
малодетный с долгом        647
многодетный без долга      349
многодетный с долгом        31
Name: children_debt, dtype: int64

In [58]:
children_debt = credit_info.pivot_table(index = 'children_debt', values='debt', aggfunc = ('count', lambda x:x.count()/len(credit_info)))
children_debt.set_axis(['Доля', 'Количество'], axis='columns',inplace=True)
children_debt['Количество'] = children_debt['Количество'].astype('int')
children_debt

,Доля,Количество
children_debt,,
бездетный без долга,0.607253,13028
бездетный с долгом,0.049548,1063
малодетный без долга,0.295330,6336
малодетный с долгом,0.030158,647
многодетный без долга,0.016267,349
многодетный с долгом,0.001445,31


In [59]:
"Доля должников среди бездетных: {:.2%}".format(1063/(13028+1063))

'Доля должников среди бездетных: 7.54%'

In [60]:
"Доля должников среди малодетных: {:.2%}".format(647/(6336+647))

'Доля должников среди малодетных: 9.27%'

In [61]:
"Доля должников среди многодетных: {:.2%}".format(31/(349+31))

'Доля должников среди многодетных: 8.16%'

### Вывод

Доля должников среди клиентов с детьми выше, чем у клиентов без детей:
- на 1.73% у малодетных
- на 0.62% у многодетны

Наличие детей увеличивает финансовую нагрузку на клиентов, что может увеличить вероятность невыплаты кредита в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [62]:
def family_debt(row):
    if row['relationship_status_id'] == 1 and row['debt'] == 0:
        return 'в отношениях без долга'
    if row['relationship_status_id'] == 1 and row['debt'] == 1:
        return 'в отношениях с долгом'
    if row['relationship_status_id'] == 2 and row['debt'] == 0:
        return 'одинокие без долга'
    if row['relationship_status_id'] == 2 and row['debt'] == 1:
        return 'одинокие с долгом'
credit_info['family_debt'] = credit_info.apply(family_debt, axis=1)
credit_info['family_debt'].value_counts()

в отношениях без долга    15171
одинокие без долга         4542
в отношениях с долгом      1319
одинокие с долгом           422
Name: family_debt, dtype: int64

In [63]:
family_debt = credit_info.pivot_table(index = 'family_debt', values='debt', aggfunc = ('count', lambda x:x.count()/len(credit_info)))
family_debt.set_axis(['Доля', 'Количество'], axis='columns',inplace=True)
family_debt['Количество'] = family_debt['Количество'].astype('int')
family_debt

,Доля,Количество
family_debt,,
в отношениях без долга,0.707141,15171
в отношениях с долгом,0.061480,1319
одинокие без долга,0.211709,4542
одинокие с долгом,0.019670,422


In [64]:
"Доля должников среди клиентов в отношениях: {:.2%}".format(1319/(15171+1319))

'Доля должников среди клиентов в отношениях: 8.00%'

In [65]:
"Доля должников среди одиноких клиентов: {:.2%}".format(422/(4542+422))

'Доля должников среди одиноких клиентов: 8.50%'

### Вывод

Доля должников среди одиноких клиентов выше, чем среди клиентов в отношениях, на 0.5%. Это может говорить о том, что людям в отношении проще выплачивать кредит, т.к. они могу получить финансовую поддержку от партнёра.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [66]:
def income_debt(row):
    if row['income_level_id'] == 1 and row['debt'] == 0:
        return 'низкий доход без долга'
    if row['income_level_id'] == 1 and row['debt'] == 1:
        return 'низкий доход с долгом'
    if row['income_level_id'] == 2 and row['debt'] == 0:
        return 'средних доход без долга'
    if row['income_level_id'] == 2 and row['debt'] == 1:
        return 'средних доход с долгом'
    if row['income_level_id'] == 3 and row['debt'] == 0:
        return 'высокий доход без долга'
    if row['income_level_id'] == 3 and row['debt'] == 1:
        return 'высокий доход с долгом'
    if row['income_level_id'] == 4 and row['debt'] == 0:
        return 'миллионер без долга'
    if row['income_level_id'] == 4 and row['debt'] == 1:
        return 'миллионер с долгом'
credit_info['income_debt'] = credit_info.apply(income_debt, axis=1)
credit_info['income_debt'].value_counts()

высокий доход без долга    13045
средних доход без долга     5888
высокий доход с долгом      1153
низкий доход без долга       757
средних доход с долгом       537
низкий доход с долгом         49
миллионер без долга           23
миллионер с долгом             2
Name: income_debt, dtype: int64

In [67]:
income_debt = credit_info.pivot_table(index = 'income_debt', values='debt', aggfunc = ('count', lambda x:x.count()/len(credit_info)))
income_debt.set_axis(['Доля', 'Количество'], axis='columns',inplace=True)
income_debt['Количество'] = income_debt['Количество'].astype('int')
income_debt

,Доля,Количество
income_debt,,
высокий доход без долга,0.608045,13045
высокий доход с долгом,0.053743,1153
миллионер без долга,0.001072,23
миллионер с долгом,0.000093,2
низкий доход без долга,0.035285,757
низкий доход с долгом,0.002284,49
средних доход без долга,0.274448,5888
средних доход с долгом,0.025030,537


In [68]:
"Доля должников среди клиентов c низким доходом: {:.2%}".format(49/(757+49))

'Доля должников среди клиентов c низким доходом: 6.08%'

In [69]:
"Доля должников среди клиентов cо средним доходом: {:.2%}".format(537/(5888+537))

'Доля должников среди клиентов cо средним доходом: 8.36%'

In [70]:
"Доля должников среди клиентов c высоким доходом: {:.2%}".format(1153/(13045+1153))

'Доля должников среди клиентов c высоким доходом: 8.12%'

In [71]:
"Доля должников среди клиентов миллионеров: {:.2%}".format(2/(23+2))

'Доля должников среди клиентов миллионеров: 8.00%'

### Вывод

Доля должников среди большинства групп равна ~8%. Кроме группы с низким доходом: у них 6.08%. На основании этих данных можно предположить, что зависимости между уровнем дохода и возвратом кредита в срок нет.

- Как разные цели кредита влияют на его возврат в срок?

In [72]:
def purpose_debt(row):
    if row['purpose_type_id'] == 1 and row['debt'] == 0:
        return 'недвижимость без долга'
    if row['purpose_type_id'] == 1 and row['debt'] == 1:
        return 'недвижимость с долгом'
    if row['purpose_type_id'] == 2 and row['debt'] == 0:
        return 'автомобиль без долга'
    if row['purpose_type_id'] == 2 and row['debt'] == 1:
        return 'автомобиль с долгом'
    if row['purpose_type_id'] == 3 and row['debt'] == 0:
        return 'образование без долга'
    if row['purpose_type_id'] == 3 and row['debt'] == 1:
        return 'образование с долгом'
    if row['purpose_type_id'] == 4 and row['debt'] == 0:
        return 'свадьба без долга'
    if row['purpose_type_id'] == 4 and row['debt'] == 1:
        return 'свадьба с долгом'
credit_info['purpose_debt'] = credit_info.apply(purpose_debt, axis=1)
credit_info['purpose_debt'].value_counts()

недвижимость без долга    10029
автомобиль без долга       3903
образование без долга      3643
свадьба без долга          2138
недвижимость с долгом       782
автомобиль с долгом         403
образование с долгом        370
свадьба с долгом            186
Name: purpose_debt, dtype: int64

In [73]:
purpose_debt = credit_info.pivot_table(index = 'purpose_debt', values='debt', aggfunc = ('count', lambda x:x.count()/len(credit_info)))
purpose_debt.set_axis(['Доля', 'Количество'], axis='columns',inplace=True)
purpose_debt['Количество'] = purpose_debt['Количество'].astype('int')
purpose_debt

,Доля,Количество
purpose_debt,,
автомобиль без долга,0.181924,3903
автомобиль с долгом,0.018784,403
недвижимость без долга,0.467465,10029
недвижимость с долгом,0.036450,782
образование без долга,0.169805,3643
образование с долгом,0.017246,370
свадьба без долга,0.099655,2138
свадьба с долгом,0.008670,186


In [74]:
"Доля должников среди клиентов c целью 'недвижимость': {:.2%}".format(782/(10029+782))

"Доля должников среди клиентов c целью 'недвижимость': 7.23%"

In [75]:
"Доля должников среди клиентов c целью 'автомобиль': {:.2%}".format(403/(3903+403))

"Доля должников среди клиентов c целью 'автомобиль': 9.36%"

In [76]:
"Доля должников среди клиентов c целью 'образование': {:.2%}".format(370/(3643+370))

"Доля должников среди клиентов c целью 'образование': 9.22%"

In [77]:
"Доля должников среди клиентов c целью 'свадьба': {:.2%}".format(186/(2138+186))

"Доля должников среди клиентов c целью 'свадьба': 8.00%"

### Вывод

Наибольшая доля должников у групп 'автомобиль' и 'образование': 9.36% и 9.22%. Затем следует 'свадьба' с 8%, и 'недвижимость' с 7.23%. Можно предположить, что высокая доля должников у групп 'автомобиль' и 'образование' обусловлена возрастом клиентов: кредиты на покупку автомобиля и на получение образования в основном берёт молодёжь, у которой могут быть проблемы с доходом. Доля должников у группы 'свадьба' чуть ниже, т.к. с погашением кредита могут помогать семьи обоих молодожёнов.

### Шаг 4. Общий вывод

Кредитный отдел банка предоставил нам таблицу '/datasets/data.csv' со статистикой о платёжеспособности клиентов с целью узнать, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

В предоставленной таблице имелось множество неточностей:
- наличие дупликатов в столбцах 'children', 'gender' и 'education'
- данные в 'days_employed' и 'total_income' имеют отрицательное значение
- данные в 'days_employed' и 'total_income' имеют вид дробных чисел
- значения 'days_employed' для пенсионеров и безработных нереально высоки
- пропуски в 'days_employed' и 'total_income'

Сначала мы заполнили пропуски в столбцах 'days_employed' и 'total_income'. Пропуски в 'days_employed' были заполнены нулями, а пропуски в 'total_income' были заполнены медианами дохода каждой группы доходности 'income_type'. Были выдвинуты следующие гипотезы наличия пропусков:
- банк не смог добыть эти данные
- утеря данных связана с технической ошибкой в системе кредитного отдела банка

Далее мы изменили тип данных в колонках 'days_employed' и 'total_income' на int64, с которым проще работать.

Потом были удалены дубликаты в колонках 'children' и 'education'. Для 'children' использовался метод .replace(), т.к. нужно было заменить всего два значения: '-1' и '20'. Для 'education' использовался метод str.lower(), вернувший строки колонки к нижнему регистру. После этого был использован метод .drop_duplicates() для удаления всех дубликатов в таблице. Возможная причина появления дубликатов:
- ошибка при сборе данных клиентов.

Следующим шагом была лемматизация. Для проведения лемматизации к колонке 'purpose' была применена функция lemmatizator, а результат был записан в колонку 'purpose_lemmas'. После изучения колонки 'purpose_lemmas' были выделены 4 группы целей кредитов:
- недвижимость
- автомобиль
- образование
- свадьба
Далее была составлена функция 'purpose_type', которая проверяет колонку 'purpose_lemmas' на наличие ключевых слов, и при совпадении, определяет цель кредита к определённой группе.

Последний шаг в предобработке данных - категоризация. Мы категоризовали данные по следующим критериям:
- количество детей
- семейное положение
- уровень дохода

В конце проекта были даны ответы на поставленные вопросы:
- Есть ли зависимость между наличием детей и возвратом кредита в срок? 
Ответ: Наличие детей увеличивает финансовую нагрузку на клиентов, что может увеличить вероятность невыплаты кредита в срок.
    
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
Ответ: Доля должников среди одиноких клиентов выше, чем среди клиентов в отношениях, на 0.5%. Это может говорить о том, что людям в отношении проще выплачивать кредит, т.к. они могу получить финансовую поддержку от партнёра.
    
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Ответ: Доля должников среди большинства групп равна ~8%. Кроме группы с низким доходом: у них 6.08%. На основании этих данных можно предположить, что зависимости между уровнем дохода и возвратом кредита в срок нет.

- Как разные цели кредита влияют на его возврат в срок?
Ответ: Наибольшая доля должников у групп 'автомобиль' и 'образование': 9.36% и 9.22%. Затем следует 'свадьба' с 8%, и 'недвижимость' с 7.23%. Можно предположить, что высокая доля должников у групп 'автомобиль' и 'образование' обусловлена возрастом клиентов: кредиты на покупку автомобиля и на получение образования в основном берёт молодёж, у которой могут быть проблемы с доходом. Доля должников у группы 'свадьба' чуть ниже, т.к. с погашением кредита могут помогать семьи обоих молодожёнов.